In [13]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 39kB/s 
     |████████████████████████████████| 3.8MB 46.3MB/s 
     |████████████████████████████████| 512kB 43.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=6d2918158f165d62cd89aa0dc254ae3e32fa83aa8be3092357dc82b11438f80c
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorflow 2.2.0rc1
    Uninstalling tensorflow-2.2.0rc1:
      Successfully uninstalled tensorflow-2.2.0rc

In [1]:
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
import numpy as np
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
# File Reading
with open('/content/alice_in_wonderland.txt','r') as content_file:
  content = content_file.read()
content2 = " ".join("".join([" "if ch in string.punctuation else ch for ch in content]).split())
tokens = nltk.word_tokenize(content2)
tokens = [word.lower() for word in tokens if len(word)>=2]

In [0]:
# N value = 3 for N grams, in which N-1 are used to predict last Nth word
N = 3
quads = list(nltk.ngrams(tokens,N))
newl_app = []
for ln in quads:
  newl = " ".join(ln)
  newl_app.append(newl)

In [4]:
# Vectorizing the words
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
x_trigm = []
y_trigm = []
for l in newl_app:
  x_str = " ".join(l.split()[0:N-1])
  y_str = l.split()[N-1]
  x_trigm.append(x_str)
  y_trigm.append(y_str)
x_trigm_check = vectorizer.fit_transform(x_trigm).todense()
y_trigm_check = vectorizer.fit_transform(y_trigm).todense()
# Dictionaries from word to integer and integer to word 
dictnry = vectorizer.vocabulary_
rev_dictnry = {v:k for k,v in dictnry.items()}
X = np.array(x_trigm_check)
Y = np.array(y_trigm_check)
Xtrain, Xtest, Ytrain, Ytest,xtrain_tg,xtest_tg = train_test_split(X, Y,x_trigm, test_size=0.3,random_state=42)
print("X Train shape",Xtrain.shape, "Y Train shape" , Ytrain.shape)
print("X Test shape",Xtest.shape, "Y Test shape" , Ytest.shape)

X Train shape (17944, 2562) Y Train shape (17944, 2562)
X Test shape (7691, 2562) Y Test shape (7691, 2562)


In [7]:
# Model 
from keras.layers import Input,Dense,Dropout
from keras.models import Model
np.random.seed(42)
BATCH_SIZE = 128
NUM_EPOCHS = 100
input_layer = Input(shape = (Xtrain.shape[1],),name="input")
first_layer = Dense(1000,activation='relu',name = "first")(input_layer)
first_dropout = Dropout(0.5,name="firstdout")(first_layer)
second_layer = Dense(800,activation='relu',name="second") (first_dropout)
third_layer = Dense(1000,activation='relu',name="third") (second_layer)
third_dropout = Dropout(0.5,name="thirdout")(third_layer)
fourth_layer = Dense(Ytrain.shape[1],activation='softmax',name = "fourth")(third_dropout)
model = Model(input_layer,fourth_layer)
model.compile(optimizer = "adam",loss="categorical_crossentropy", metrics=["accuracy"])
print (model.summary())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 2562)              0         
_________________________________________________________________
first (Dense)                (None, 1000)              2563000   
_________________________________________________________________
firstdout (Dropout)          (None, 1000)              0         
_________________________________________________________________
second (Dense)               (None, 800)               800800    
_________________________________________________________________
third (Dense)                (None, 1000)              801000    
_________________________________________________________________
thirdout (Dropout)           (None, 1000)              0         
_________________________________________________________________
fourth (Dense)               (None, 2562)              2564

In [8]:
history = model.fit(Xtrain, Ytrain, batch_size=BATCH_SIZE,epochs=NUM_EPOCHS, verbose=1,validation_split = 0.2)

Train on 14355 samples, validate on 3589 samples
Epoch 1/100
14355/14355 [==============================] - 2s 159us/step - loss: 6.3265 - acc: 0.0567 - val_loss: 6.0697 - val_acc: 0.0607
Epoch 2/100
14355/14355 [==============================] - 2s 125us/step - loss: 5.9152 - acc: 0.0688 - val_loss: 6.0358 - val_acc: 0.0627
Epoch 3/100
14355/14355 [==============================] - 2s 125us/step - loss: 5.7058 - acc: 0.0791 - val_loss: 5.9856 - val_acc: 0.0853
Epoch 4/100
14355/14355 [==============================] - 2s 124us/step - loss: 5.4673 - acc: 0.1041 - val_loss: 5.9346 - val_acc: 0.0950
Epoch 5/100
14355/14355 [==============================] - 2s 126us/step - loss: 5.2131 - acc: 0.1311 - val_loss: 5.9243 - val_acc: 0.0995
Epoch 6/100
14355/14355 [==============================] - 2s 126us/step - loss: 4.9490 - acc: 0.1524 - val_loss: 5.9567 - val_acc: 0.1064
Epoch 7/100
14355/14355 [==============================] - 2s 126us/step - loss: 4.7266 - acc: 0.1767 - val_loss: 5.9

In [0]:
model.save("Language_modeling.h5")

In [0]:
# Model Prediction
from keras.models import load_model
model = load_model('Language_modeling.h5')
Y_pred = model.predict(Xtest)

In [14]:
# Sample check on Test data
print ("Prior bigram words", "|Actual", "|Predicted","\n")
for i in range(50):
  print (i,xtest_tg[i],"|",rev_dictnry[np.argmax(Ytest[i])], "|",rev_dictnry[np.argmax(Y_pred[i])])

Prior bigram words |Actual |Predicted 

0 and they | re | repeated
1 never learnt | it | left
2 matters it | how | good
3 as she | could | could
4 the duchess | what | and
5 pair of | boots | white
6 cattle in | the | the
7 it was | looking | the
8 what else | have | you
9 voice of | the | thunder
10 to make | out | out
11 royal children | there | and
12 direction in | which | the
13 in the | lap | distance
14 offer it | said | was
15 of onions | seven | course
16 balanced an | eel | an
17 tremble alice | looked | and
18 forwards each | time | case
19 and will | talk | have
20 the queen | can | and
21 thing is | to | to
22 had nothing | yet | else
23 tell me | please | the
24 to get | out | in
25 to say | word | it
26 become of | me | it
27 not attending | said | to
28 about in | the | the
29 ran the | faster | of
30 the king | sharply | and
31 turning purple | won | to
32 had quite | long | to
33 was lying | on | under
34 up again | let | with
35 ve had | nothing | oh
36 on just | as 